In [1]:
from matplotlib import pyplot as plt
import numpy as np
from figure import _plot_eeg, plot_eeg, plot_sig_
from eeg_v2 import EEG
from utils import chunks, baseline_calc, butter_bandpass_filter
import joblib
from mne.preprocessing import ICA

# Function and Lib

In [2]:
def _get_ica_map(ica, components=None):
    """Get ICA topomap for components"""
    fast_dot = np.dot
    if components is None:
        components = list(range(ica.n_components_))
    maps = fast_dot(ica.mixing_matrix_[:, components].T,
                    ica.pca_components_[:ica.n_components_])
    return maps

def get_chunks(sig):
    i = 0
    for task in sig.tasks:
        for c in chunks(task, 15):
            i+=1
            yield i, c
            
def count_chunks(sig):
    i = 0
    for task in sig.tasks:
        for c in chunks(task, 15):
            i+=1
    return i

In [3]:
# Reading raw data
path_edf="./edf/1578_alice/edf/A0001578.edf"
path_stage="./edf/1578_alice/csv/STAGE.csv"

eeg = EEG(path_edf=path_edf, path_stage=path_stage)

/home/tudo/Desktop/code/ica_paper/eeg_v2.py:17: RuntimeWarning: Channel names are not unique, found duplicates for: {'EMG Chin'}. Applying running numbers for duplicates.
  raw = read_raw_edf(self.path_edf, preload=True, verbose=0)


In [4]:
count_chunks(eeg)

56

In [5]:
# # Pick a chunks

# _chunks = get_chunks(eeg)
# for i in range(16):
#     _, chunk = next(_chunks)

# print(_)
# print(np.min(chunk),np.max(chunk))

In [6]:
# Preprocessing pipeline
def baseline_bandpass(chunk):
    sample_raw = eeg.raw.copy().crop(np.min(chunk),np.max(chunk),include_tmax=False)
    sample_raw_eog = eeg.eog_channels.copy().crop(np.min(chunk),np.max(chunk),include_tmax=False)


    # Baseline
    sample_raw_baseline = sample_raw.copy()
    sample_raw_baseline = sample_raw_baseline.apply_function(baseline_calc)

    # Bandpass
    sample_raw_bandpass = sample_raw_baseline.copy()
    sample_raw_bandpass = sample_raw_bandpass.apply_function(butter_bandpass_filter)
    
    return sample_raw, sample_raw_baseline, sample_raw_bandpass, sample_raw_eog

In [7]:
# ICA pipeline
def ica_pipe(sample_raw_bandpass):    
    clf = joblib.load("./models/eog_classifier_v2.joblib")


    sample_raw_train = sample_raw_bandpass.copy()
    sample_raw_corrected = sample_raw_bandpass.copy()

    # Fitting ICA
    ica = ICA(method="extended-infomax", random_state=1)
    ica.fit(sample_raw_corrected)

    maps = _get_ica_map(ica).T
    scalings = np.linalg.norm(maps, axis=0)
    maps /= scalings[None, :]
    X = maps.T

    # Predict EOG
    eog_preds = clf.predict(X)
    list_of_eog = np.where(eog_preds == 1)[0]
    
    # ica.plot_sources(inst=sample_raw_train) 
    # ica.plot_components(inst=sample_raw_train)

    ica.exclude = list_of_eog
    ica.apply(sample_raw_corrected)
    
    return ica, sample_raw_train, sample_raw_corrected

# FIGURE 2

In [8]:
plt.rcParams["figure.figsize"] = [10,5]
PATH = "figures/figure2"
i = 0
for task in eeg.tasks:
    for c in chunks(task, 15):
        i+=1
        # if i > 1:
        #    break
        
        sample_raw, sample_raw_baseline, sample_raw_bandpass, sample_raw_eog = baseline_bandpass(c)
        # ica, sample_raw_train, sample_raw_corrected = ica_pipe(sample_raw_bandpass)
        

        # Raw
        plot_sig_(
            sig= sample_raw.get_data() * 1e3,
            eog= sample_raw_eog.get_data() * 1e3,
            ch_names = sample_raw.ch_names,
            eog_ch_names = sample_raw_eog.ch_names,
            fs=500,
            unit="10^-6 V",
            eog_unit = "10^-6 V",
            dpi=100,
            fname=f"{PATH}/fig2_raw_{i}"
        );


        # Baseline + Bandpass
        plot_sig_(
            sig= sample_raw_bandpass.get_data() * 1e3,
            eog= sample_raw_eog.get_data() * 1e3,
            ch_names = sample_raw.ch_names,
            eog_ch_names = sample_raw_eog.ch_names,
            fs=500,
            unit="10^-6 V",
            eog_unit = "10^-6 V",
            dpi=100,
            fname=f"{PATH}/fig2_bandpass_{i}"
        );

findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.


# FIGURE 3

In [9]:
i = 0
PATH = "figures/figure3"
for task in eeg.tasks:
    for c in chunks(task, 15):
        i+=1
        # if i > 1:
        # break
            
        sample_raw, sample_raw_baseline, sample_raw_bandpass, sample_raw_eog = baseline_bandpass(c)
        ica, sample_raw_train, sample_raw_corrected = ica_pipe(sample_raw_bandpass)
        
        # Save fig
        plot_sig_(
            sig= ica.get_sources(inst=sample_raw_train).get_data() * 1e3,
            eog= sample_raw_eog.get_data() * 1e3,
            ch_names = ica.get_sources(inst=sample_raw_train).ch_names,
            eog_ch_names = sample_raw_eog.ch_names,
            fs=500,
            # unit="10^-6 V",
            eog_unit = "10^-6 V",
            dpi=100,
            fname=f"{PATH}/fig3_{i}"
        );

Fitting ICA to data using 10 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 10
Computing Extended Infomax ICA


<ipython-input-7-9f71c3044bd4>:10: DeprecationWarning: method='extended-infomax' is deprecated and will be removed in 0.19. If you want to use Extended Infomax, specify method='infomax' together with fit_params=dict(extended=True).
  ica = ICA(method="extended-infomax", random_state=1)


Fitting ICA took 2.5s.
Transforming to ICA space (10 components)
Zeroing out 2 ICA components
Fitting ICA to data using 10 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 10
Computing Extended Infomax ICA


<ipython-input-7-9f71c3044bd4>:10: DeprecationWarning: method='extended-infomax' is deprecated and will be removed in 0.19. If you want to use Extended Infomax, specify method='infomax' together with fit_params=dict(extended=True).
  ica = ICA(method="extended-infomax", random_state=1)


Fitting ICA took 2.0s.
Transforming to ICA space (10 components)
Zeroing out 1 ICA components
Fitting ICA to data using 10 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 10
Computing Extended Infomax ICA


...

Fitting ICA took 1.7s.
Transforming to ICA space (10 components)
Zeroing out 2 ICA components
Fitting ICA to data using 10 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 10
Computing Extended Infomax ICA


<ipython-input-7-9f71c3044bd4>:10: DeprecationWarning: method='extended-infomax' is deprecated and will be removed in 0.19. If you want to use Extended Infomax, specify method='infomax' together with fit_params=dict(extended=True).
  ica = ICA(method="extended-infomax", random_state=1)


Fitting ICA took 2.1s.
Transforming to ICA space (10 components)
Zeroing out 2 ICA components
Fitting ICA to data using 10 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 10
Computing Extended Infomax ICA


<ipython-input-7-9f71c3044bd4>:10: DeprecationWarning: method='extended-infomax' is deprecated and will be removed in 0.19. If you want to use Extended Infomax, specify method='infomax' together with fit_params=dict(extended=True).
  ica = ICA(method="extended-infomax", random_state=1)


Fitting ICA took 1.9s.
Transforming to ICA space (10 components)
Zeroing out 1 ICA components
